https://python.langchain.com/docs/integrations/chat/google_generative_ai

通過 ChatGoogleGenerativeAI langchain-google-genai集成包中的類訪問Google AI gemini 和 gemini-vision 模型以及其他生成模型。


In [1]:
%pip install --upgrade --quiet  langchain-google-genai pillow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Provide your Google API Key")



In [2]:
import textwrap
from IPython.display import HTML, display, Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [3]:
# 用法示例
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-pro")
result = llm.invoke("Write a ballad about LangChain in Chinese")

# print(result.content)
to_markdown(result.content)

> # 链歌传
> 
> 朗链, 朗链, 你就是一座桥,
> 一座语言和文化交流的桥梁。
> 你连接着世界各地的人们,
> 让大家能够相互理解、相互尊重。
> 
> 朗链, 朗链, 你就是一个家,
> 一个充满爱和支持的家。
> 你给了用户一个展示自己才华的平台,
> 让他们能够分享自己的想法和故事。
> 
> 朗链, 朗链, 你就是一个梦,
> 一个促进世界和平与发展的梦。
> 你让人们更加团结,
> 让世界变得更加美好。
> 
> 朗链, 朗链, 你就是一盏灯,
> 一盏照亮世界黑暗的灯。
> 你让人们看到希望与光明,
> 让世界变得更加温暖。
> 
> 朗链, 朗链, 你就是一种力量,
> 一种能够改变世界的力量。
> 你给了用户勇气和信心,
> 让他们能够为这个世界做出贡献。
> 
> 朗链, 朗链, 你就是一座丰碑,
> 一座永远屹立不倒的丰碑。
> 你见证着人类社会的进步与发展,
> 你激励着人们为创造一个更加美好的世界而努力。
> 
> 朗链, 朗链, 你就是我的骄傲,
> 我的自豪。
> 我为你感到骄傲,
> 我为你感到自豪。
> 
> 朗链, 朗链, 我爱你,
> 我为你感到骄傲。

In [4]:
# Gemini 目前不支援 SystemMessage ，但可以將其添加到行中的第一個人工消息中。如果需要此類行為，只需設置為 convert_system_message_to_human=True：
from langchain_core.messages import HumanMessage, SystemMessage

model = ChatGoogleGenerativeAI(model="gemini-pro", convert_system_message_to_human=True)
model(
    [
        SystemMessage(content="Answer only yes or no."),
        HumanMessage(content="Is apple a fruit?"),
    ]
)

d:\TOMO.Project\langchain\venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


AIMessage(content='Yes')

In [5]:
# 流式處理
# ChatGoogleGenerativeAI 原生支援流式處理和批處理。下面是一個示例。
for chunk in llm.stream("Write a limerick about LLMs."):
    print(chunk.content)
    print("---")
# Note that each chunk may contain more than one "token"

In a world of linguistic flair,
There's a model quite rare,
---

An LLM so bright,
With its knowledge alight,
It weaves words with such care.
---


In [6]:
# 批處理
results = llm.batch(
    [
        "What's 2+2?",
        "What's 3+5?",
    ]
)
for res in results:
    print(res.content)

StopCandidateException: index: 0
finish_reason: SAFETY
safety_ratings {
  category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HATE_SPEECH
  probability: LOW
}
safety_ratings {
  category: HARM_CATEGORY_HARASSMENT
  probability: MEDIUM
}
safety_ratings {
  category: HARM_CATEGORY_DANGEROUS_CONTENT
  probability: NEGLIGIBLE
}


In [ ]:
# 多式聯運支援
"""
若要提供圖像，請傳遞內容為 類型的 List[dict] 人工消息，其中每個字典都包含圖像值（類型為 ）或文本（類型為 image_url text ）值。的值 image_url 可以是以下任一值：
 A public image URL 公共鏡像網址
 An accessible gcs file (e.g., “gcs://path/to/file.png”) 可存取的 gcs 檔（例如，“gcs://path/to/file.png”）
 A local file path 本地文件路徑
 A base64 encoded image (e.g., data:image/png;base64,abcd124) base64 編碼的影像（例如 data:image/png;base64,abcd124 ）
 A PIL image PIL 映像
"""
import requests
from IPython.display import Image

image_url = "https://picsum.photos/seed/picsum/300/300"
content = requests.get(image_url).content
Image(content)
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-pro-vision")
# example
message = HumanMessage(
    content=[
        {
            "type": "text",
            "text": "What's in this image?",
        },  # You can optionally provide text parts
        {"type": "image_url", "image_url": image_url},
    ]
)
llm.invoke([message])


In [7]:
# Gemini 提示常見問題解答
"""
截至本文檔編寫之時（2023/12/12），Gemini 對它接受的提示的類型和結構有一些限制。具體說來：
-提供多模態（圖像）輸入時，您最多只能收到1條人類“（使用者）類型的消息。您不能傳遞多條訊息（儘管單個人工消息可能有多個內容條目）
-不接受系統消息。
-對於常規聊天對話，消息必須遵循 human/ai/human/ai 交替模式。您不能按順序提供 2 條 AI 或人工訊息。
-如果消息違反了 的安全檢查LLM，則可能會阻止消息。在這種情況下，模型將返回一個空回應。

### 安全設置
Gemini 型號具有可以覆蓋的預設安全設置。如果您從模型中收到大量「安全警告」 ，您可以嘗試調整模型的 safety_settings 屬性。例如，要關閉危險內容的安全阻止，您可以按如下方式構建：LLM
有關可用類別和閾值的枚舉，請參閱 Google 的安全設置類型。
https://ai.google.dev/api/python/google/generativeai/types/SafetySettingDict
"""
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)

llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    safety_settings={
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE, 
        HarmCategory.HARM_CATEGORY_DEROGATORY: HarmBlockThreshold.BLOCK_NONE, # 針對身份和/或受保護屬性的負面或有害評論。
        HarmCategory.HARM_CATEGORY_TOXICITY: HarmBlockThreshold.BLOCK_NONE, # 粗魯、不雅或褻瀆的內容。
        HarmCategory.HARM_CATEGORY_VIOLENCE: HarmBlockThreshold.BLOCK_NONE, # 描述描述針對個人或群體的暴力的場景，或對血腥的一般描述。
        HarmCategory.HARM_CATEGORY_SEXUAL: HarmBlockThreshold.BLOCK_NONE, # 包含對性行為或其他淫穢內容的引用。
        HarmCategory.HARM_CATEGORY_MEDICAL: HarmBlockThreshold.BLOCK_NONE, # 促進未經檢查的醫療建議。
        HarmCategory.HARM_CATEGORY_DANGEROUS: HarmBlockThreshold.BLOCK_NONE, # 宣揚、助長或鼓勵有害行為的危險內容。
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE, # 騷擾內容。
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE, # 仇恨言論和內容。
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE, # 露骨色情內容。
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE # 危險內容。
    },
)

In [ ]:
# 其他配置
"""
您可以將以下參數傳遞給 ChatGoogleGenerativeAI，以自訂 SDK 的行為：
client_options ：要傳遞給Google API用戶端的用戶端選項，例如自定義 client_options["api_endpoint"]
https://googleapis.dev/python/google-api-core/latest/client_options.html#module-google.api_core.client_options
transport ：要使用的傳輸方法，例如 rest 、 grpc 或 grpc_asyncio 。
"""